***

### Logic
#### Backlog giao: Bưu cục nhận hàng giao từ tải => Giao thành công/Trả hàng
    * Không quá 120h.
#### Backlog trả: Bưu cục nhận hàng trả từ tải => Trả hàng thành công
    * Không quá 72h.
#### Backlog lấy
    * Không quá 72h?
#### Backlog luân chuyển lấy: Lấy hàng thành công => Bàn giao tải 
    * HCM -> HN: 12h.
    * Các tỉnh còn lại: 32h.
#### Backlog luân chuyển trả: Chuyển trạng thái trả hàng => Bàn giao tải
    * Giao hàng tại HCM HN: 36h.
    * Giao hàng tại các tỉnh còn lại: 56h.
***

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import pandas as pd
import numpy as np
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as df2g
import os


# change current working directory to Downloads
os.chdir('/Users/phuoc/Downloads')

# read data
export = pd.read_excel('export.xlsx')
inside = pd.read_excel('inside.xlsx')

# old_name: new_name -> replace inside headers
renamed_inside_headers = {
    'Mã đơn': 'MaDH',
    'Mã kiện': 'MaKien',
    'Kho gửi': 'KhoGui',
    'Kho nhận': 'KhoNhan',
    'Kho hiện tại': 'KhoHienTai',
    'TG đóng kiện': 'TGDongKien',
    'TG cập nhật': 'TGCapNhat',
    'TG nhận kiện': 'TGNhanKien',
    'TG kết thúc': 'TGKetThuc',
    'Trạng thái': 'TrangThaiLuanChuyen',
    'Số đơn': 'SoDon',
    'Khối lượng': 'KhoiLuong',
    'Mã niêm phong đóng': 'MaNiemPhongDong',
    'Mã niêm phong nhận': 'MaNiemPhongNhan',
    'Hình thức đóng gói': 'HinhThucDongGoi',
    'Hình thức vận chuyển': 'HinhThucVanChuyen',
    'Ghi chú': 'GhiChu',
}

# rename columns name of inside dataframe
inside = inside.rename(columns=renamed_inside_headers)

# normalize datetime of inside dataframe
# inside['TGDongKien'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))

# change str to datetime
inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]] = inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y %H:%M:%S'))
inside['KhoGui'] = inside['KhoGui'].astype(int)
inside['KhoNhan'] = inside['KhoNhan'].astype(int)
inside['KhoHienTai'] = inside['KhoHienTai'].astype(int)

# normalize datetime of export dataframe
# replace all <nil> values
export = export.replace('<nil>', np.nan)

# change str to datetime
export[[
    'ThoiGianTao',
    'ThoiGianTaoChuyenDoi',
    'ThoiGianKetThucLay',
    'ThoiGianGiaoLanDau',
    'ThoiGianKetThucGiao',
    'ThoiGianGiaoHangMongMuon',
    'TGKetThucTra',
]] = export[[
        'ThoiGianTao',
        'ThoiGianTaoChuyenDoi',
        'ThoiGianKetThucLay',
        'ThoiGianGiaoLanDau',
        'ThoiGianKetThucGiao',
        'ThoiGianGiaoHangMongMuon',
        'TGKetThucTra',
]].apply(pd.to_datetime)

# set display of dataframe
pd.set_option('display.max_columns', 88)
pd.set_option('display.max_rows', 100)

# join export with inside
data = pd.merge(export, inside[['MaDH','MaKien', 'KhoGui', 'KhoNhan', 'TGNhanKien', 'TrangThaiLuanChuyen']], on='MaDH', how='left')

# codes of e-commerces exchange
shopee_codes = ['18692']
sendo_codes = ['1539', '1160902', '1160904', '1160905']
tiki_codes = ['1367']
lazada_codes = ['1041351', '9794']
ecoms = shopee_codes + sendo_codes + tiki_codes + lazada_codes

# classify types of backlog
# delivery backlog (backlog giao)
delivery_filter = data['TrangThai'].isin([
    'Đang giao hàng', 'Giao hàng không thành công'
]) | (((data['TrangThai'] == 'Lưu kho') | (data['TrangThai'] == 'Lấy hàng thành công') | (data['TrangThai'] == 'Đang trung chuyển hàng')) &\
      (data['KhoGiao'] == data['KhoHienTai'])) | ((data['TrangThai'] == 'Chờ giao lại') & (datetime.now() - data['ThoiGianKetThucGiao'] > pd.Timedelta('24 hours')))

delivery = data.loc[delivery_filter]
delivery['LoaiBacklog'] = 'Kho giao'
internal_delivery_filter = delivery['KhoLay'] == delivery['KhoHienTai']
delivery.loc[internal_delivery_filter, 'N0'] = delivery.loc[internal_delivery_filter]['ThoiGianKetThucLay']
delivery.loc[~internal_delivery_filter, 'N0'] = delivery.loc[~internal_delivery_filter]['TGNhanKien']
delivery['N+'] = delivery['N0'] + pd.Timedelta(hours=120)
delivery['Aging'] = (datetime.today() - delivery['N+']).fillna(pd.Timedelta(hours=9999))


# return backlog (backlog trả)
returned_filter = data['TrangThai'].isin([
    'Đang hoàn hàng', 'Hoàn hàng không thành công'
    ]) | (((data['TrangThai'] == 'Chuyển hoàn') | (data['TrangThai'] == 'Đang trung chuyển hàng hoàn')) & (((~data['KhoTra'].isnull()) & (data['KhoTra'] == data['KhoHienTai'])) |\
                                                 ((data['KhoTra'].isnull()) & (data['KhoLay'] == data['KhoHienTai']))))
returned = data.loc[returned_filter]
returned['LoaiBacklog'] = 'Kho trả'
internal_return_filter = returned['KhoHienTai'] == returned['KhoGiao']
returned.loc[internal_return_filter, 'N0'] = returned.loc[internal_return_filter]['ThoiGianKetThucGiao']
returned.loc[~internal_return_filter, 'N0'] = returned.loc[~internal_return_filter]['TGNhanKien']
returned['N+'] = returned['N0'] + pd.Timedelta(hours=72)
returned['Aging'] = (datetime.today() - returned['N+']).fillna(pd.Timedelta(hours=9999))


# pickup backlog (backlog lấy)
pickup_filter = (data['TrangThai'].isin(['Chờ lấy hàng', 'Đang lấy hàng', 'Lấy không thành công'])) |\
              ((data['TrangThai'] == 'Tạo thành công') & (data['ThoiGianKetThucLay'].isnull()) &\
               (data['KhoHienTai'] == data['KhoLay']))
pickup = data.loc[pickup_filter]
pickup['LoaiBacklog'] = 'Kho lấy'
ecoms_filter = pickup['MaKH'].isin(ecoms)
pickup.loc[ecoms_filter, 'N0'] = pickup.loc[ecoms_filter]['ThoiGianTaoChuyenDoi']
pickup.loc[~ecoms_filter, 'N0'] = pickup.loc[~ecoms_filter]['ThoiGianTao']
pickup['N+'] = pickup['N0'] + pd.Timedelta(hours=72)
pickup['Aging'] = (datetime.today() - pickup['N+']).fillna(pd.Timedelta(hours=9999))

# transporting backlog (backlog luân chuyển)
trans_filter = (((data['TrangThai'] == 'Lấy hàng thành công') | (data['TrangThai'] == 'Lưu kho')) & (data['KhoGiao'] != data['KhoHienTai'])) |\
                       ((data['TrangThai'] == 'Đang trung chuyển hàng') & (data['KhoLay'] != data['KhoGiao']))
transporting = data.loc[trans_filter]
transporting['LoaiBacklog'] = 'Kho lấy luân chuyển'
transporting_filter = (transporting['DenTinh'] == 'Hà Nội') | (transporting['DenTinh'] == 'Hồ Chí Minh')
transporting['N0'] = transporting['ThoiGianKetThucLay']
transporting.loc[transporting_filter, 'N+'] = transporting.loc[transporting_filter]['N0']+ pd.Timedelta(hours=12)
transporting.loc[~transporting_filter, 'N+'] = transporting.loc[~transporting_filter]['N0'] + pd.Timedelta(hours=32)
transporting['Aging'] = (datetime.today() - transporting['N+']).fillna(pd.Timedelta(hours=9999))


# returned_transporting backlog (backlog luân chuyển trả hàng)
rt_filter = ((data['TrangThai'].isin(['Chuyển hoàn', 'Chờ chuyển hoàn'])) & (((data['KhoTra'].isnull()) & (data['KhoLay'] != data['KhoHienTai'])) | ((~data['KhoTra'].isnull()) & (data['KhoTra'] != data['KhoHienTai'])))) |\
        ((data['TrangThai'] == 'Đang trung chuyển hàng hoàn') & (((data['KhoTra'].isnull()) & (data['KhoLay'] != data['KhoHienTai'])) | ((~data['KhoTra'].isnull()) & (data['KhoTra'] != data['KhoHienTai']))))
r_transporting = data.loc[rt_filter]
r_transporting['LoaiBacklog'] = 'Kho giao luân chuyển'
r_transporting_filter = (r_transporting['DenTinh'] == 'Hà Nội') | (r_transporting['DenTinh'] == 'Hồ Chí Minh')
r_transporting['N0'] = r_transporting['ThoiGianKetThucGiao']
r_transporting.loc[r_transporting_filter, 'N+'] = r_transporting.loc[r_transporting_filter]['N0'] + pd.Timedelta(hours=36)
r_transporting.loc[~r_transporting_filter, 'N+'] = r_transporting.loc[~r_transporting_filter]['N0'] + pd.Timedelta(hours=56)
r_transporting['Aging'] = (datetime.today() - r_transporting['N+']).fillna(pd.Timedelta(hours=9999))


# composite data
data = delivery.append([returned, pickup, transporting, r_transporting])
# set e-commerces column to NaN
data['ECommerces'] = np.nan


# name of e-commerces exchange
# shopee
data.loc[data['MaKH'].isin(shopee_codes), 'ECommerces'] = 'Shopee'
# sendo
data.loc[data['MaKH'].isin(sendo_codes), 'ECommerces'] = 'Sendo'
# tiki
data.loc[data['MaKH'].isin(tiki_codes), 'ECommerces'] = 'Tiki'
# lazada
data.loc[data['MaKH'].isin(lazada_codes), 'ECommerces'] = 'Lazada'

# ToanTrinh
data['Aging_ToanTrinh'] = np.nan
ecoms_filter = data['MaKH'].isin(ecoms)
data.loc[ecoms_filter, 'Aging_ToanTrinh'] = datetime.today() - data[ecoms_filter]['ThoiGianTaoChuyenDoi']
data.loc[~ecoms_filter, 'Aging_ToanTrinh'] = datetime.today() - data[~ecoms_filter]['ThoiGianTao']
# reset the index of datafram
data.reset_index(inplace=True)
del data['index']
data['KhoGui'].astype('int64', errors='ignore')
data['NgayHienTai'] = datetime.today().strftime('%Y-%m-%d')
data
# data['KhoGui']

,MaDH,MaDHShop,ThoiGianTao,ThoiGianTaoChuyenDoi,ThoiGianKetThucLay,ThoiGianGiaoLanDau,ThoiGianKetThucGiao,ThoiGianGiaoHangMongMuon,TGKetThucTra,ThongTinTraHang,TuQuan,TuTinh,TuVung,DenQuan,DenTinh,DenVung,KhoLay,KhoGiao,KhoHienTai,KhoTra,TrangThai,TrongLuong,KichThuoc,MaKH,GhiChuGHN,GhiChu,GoiCuoc,SoLanLay,SoLanGiao,SoLanTra,TinhTrangLuanChuyen,MaNVLay,MaNVGiao,TenNguoiGui,MaDiaChiLay,MaKien,KhoGui,KhoNhan,TGNhanKien,TrangThaiLuanChuyen,LoaiBacklog,N0,N+,Aging,ECommerces,Aging_ToanTrinh,NgayHienTai
0,7FGQS7FD3Y,NaN,2020-07-31 19:05:20,2020-08-01 08:00:00,2020-08-01 12:39:13,2020-08-29 08:19:49,2020-08-29 10:37:50,2020-08-06 01:06:00,NaT,'KHÁCH TỪ CHỐI NHẬN/Khách đổi ý',Huyện Đan Phượng,Hà Nội,G,Quận 4,Hồ Chí Minh,E,Bưu Cục 8 Đình Nguyên-Đan Phượng-HN,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 8 Đình Nguyên-Đan Phượng-HN,Chờ giao lại,15000,10x10x10,813308,'08/08/2020 02:43:24 - GHN-DCD1A3|KHÁCH TỪ CHỐ...,"'HÀNG DỄ VỠ XIN NHẸ TAY, KHÁCH KHÔNG NHẬN HÀNG...",2 Ngày,1,1,0,NaN,1874485,NaN,'Vân Trần',811421,20FFL27873,2268.0,2275.0,2020-08-29 02:57:20,Kết thúc,Kho giao,2020-08-29 02:57:20,2020-09-03 02:57:20,-3 days +14:36:26.996709,NaN,30 days 22:28:27.387993,2020-08-31
1,FHPAFQS4,KV1293243226366,2020-08-01 10:57:01,2020-08-01 10:57:00,2020-08-01 16:10:36,2020-08-04 09:17:38,NaT,2020-08-04 23:00:00,NaT,NaN,Quận Hoàng Mai,Hà Nội,G,Quận 7,Hồ Chí Minh,E,Bưu Cục 25 Trương Định-Q.Hai Bà Trưng-HN 02,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,NaN,Đang giao hàng,500,1x1x1,518873,NaN,NaN,2 Ngày,1,1,0,0,1714228,9999,'ptlink',2267105,20NRU94012,2329.0,2275.0,2020-08-03 14:07:25,Kết thúc,Kho giao,2020-08-03 14:07:25,2020-08-08 14:07:25,23 days 03:26:21.996709,NaN,30 days 06:36:46.387993,2020-08-31
2,FH7DRSYUF,INV/2020/10394,2020-08-12 21:49:43,2020-08-13 08:00:00,2020-08-13 20:54:00,NaT,NaT,2020-08-17 23:00:00,NaT,NaN,Quận Bắc Từ Liêm,Hà Nội,G,Quận 7,Hồ Chí Minh,E,Bưu Cục Phú Diễn-Q.Bắc Từ Liêm-HN,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,NaN,Lưu kho,1000,10x10x10,1298385,NaN,"'1 Tổng Thu COD; Cho xem hàng , không cho thử'",2 Ngày,1,NaN,0,0,1872479,NaN,'Shop Diệu Hân',3336737,20XCT63012,2329.0,2275.0,2020-08-16 14:12:35,Kết thúc,Kho giao,2020-08-16 14:12:35,2020-08-21 14:12:35,10 days 03:21:11.996709,NaN,18 days 19:44:04.387993,2020-08-31
3,8FHX4Q19R7,2416068723798016203,2020-08-14 14:33:00,2020-08-14 14:33:00,2020-08-15 11:49:31,NaT,NaT,2020-08-18 08:44:00,NaT,NaN,Quận Tân Phú,Hồ Chí Minh,E,Quận 7,Hồ Chí Minh,E,Bưu Cục 20/82-84 Bờ Bao Tân Thắng-Q.Tân Phú-HCM,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 20/82-84 Bờ Bao Tân Thắng-Q.Tân Phú-HCM,Lưu kho,2000,10x10x10,18692,'14/08/2020 08:19:42 - GHN-PFA1A0|THAY ĐỔI THÔ...,'Không cho xem hàng. Khối lượng nhận hàng tối ...,1 Ngày,2,0,0,NaN,253168,NaN,'Cửa hàng Trung Nghĩa',1154025,20MPV10687,2329.0,2275.0,2020-08-16 01:25:19,Kết thúc,Kho giao,2020-08-16 01:25:19,2020-08-21 01:25:19,10 days 16:08:27.996709,Shopee,17 days 03:00:47.333304,2020-08-31
4,8FHDAD3H9,S1577916O18553,2020-08-11 08:28:08,2020-08-11 08:28:08,2020-08-27 20:14:16,2020-08-30 07:45:27,NaT,2020-09-01 17:45:00,NaT,NaN,Quận Hai Bà Trưng,Hà Nội,G,Quận 7,Hồ Chí Minh,E,Bưu Cục 25 Trương Định-Q.Hai Bà Trưng-HN 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 25 Trương Định-Q.Hai Bà Trưng-HN 01,Đang giao hàng,10,10x5x5,1250888,'11/08/2020 12:39:07 - GHN-PFA1A1|THAY ĐỔI THÔ...,'',2 Ngày,4,2,0,NaN,1883445,NaN,'infinity',1201037,20FGP17873,2329.0,2275.0,2020-08-29 20:32:44,Kết thúc,Kho giao,2020-08-29 20:32:44,2020-09-03 20:32:44,-4 days +21:01:02.996709,NaN,20 days 09:05:39.387993,2020-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8524,8FHY6Q9H4F,2425436433801221398,2020-08-27 12:45:00,2020-08-27 12:44:58,2020-08-27 13:42:58,2020-08-28 08:21:09,2020-08-28 13:33

In [8]:
# export data backlog to google sheets
# data export
data_compact = data[[
    'NgayHienTai',
    'MaDH',
    'KhoHienTai',
    'TrangThai',
    'GhiChuGHN',
    'SoLanLay',
    'SoLanGiao',
    'SoLanTra',
    'LoaiBacklog',
    'N0',
    'N+',
    'Aging',
    'Aging_ToanTrinh',
    'ECommerces',
    'TrangThaiLuanChuyen',
]]

# define scope of google api json keyfile
SCOPES = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

# credentials to google service account
credentials = ServiceAccountCredentials.from_json_keyfile_name('ggapi_key.json', scopes=SCOPES)
# authorize with gspread
gc = gspread.authorize(credentials)

SPREADSHEET_ID = '1_UlAs1Ed6rxl1TNM8Vv2fnEsRT61BOWTVzCMtd1hwME'  # id of sheets (on url)
WKS_NAME = 'Backlog_bot'
active_sheets = gc.open_by_key(SPREADSHEET_ID)
wks = active_sheets.worksheet(WKS_NAME)
data_upload = data_compact.loc[data_compact['Aging'] >= pd.Timedelta(hours=0)]

# update data to google sheets
wks.clear()
df2g.upload(
    data_upload,
    SPREADSHEET_ID,
    WKS_NAME,
    credentials=credentials,
    row_names=False,
)


<Worksheet 'Backlog_bot' id:181055436>

In [9]:
from df2gspread import gspread2df as g2df

df_gspread = g2df.download(gfile=SPREADSHEET_ID, wks_name=WKS_NAME, credentials=credentials, col_names=True, row_names=False)
df_gspread

,MaDH,KhoHienTai,TrangThai,GhiChuGHN,SoLanLay,SoLanGiao,SoLanTra,LoaiBacklog,N0,N+,Aging,Aging_ToanTrinh,ECommerces,TrangThaiLuanChuyen
2020-08-31,FHPAFQS4,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Đang giao hàng,nan,1,1,0,Kho giao,2020-08-03 14:07:25,2020-08-08 14:07:25,23 days 03:26:21.996709,30 days 06:36:46.387993,nan,Kết thúc
2020-08-31,FH7DRSYUF,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Lưu kho,nan,1,nan,0,Kho giao,2020-08-16 14:12:35,2020-08-21 14:12:35,10 days 03:21:11.996709,18 days 19:44:04.387993,nan,Kết thúc
2020-08-31,8FHX4Q19R7,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Lưu kho,'14/08/2020 08:19:42 - GHN-PFA1A0|THAY ĐỔI THÔ...,2,0,0,Kho giao,2020-08-16 01:25:19,2020-08-21 01:25:19,10 days 16:08:27.996709,17 days 03:00:47.333304,Shopee,Kết thúc
2020-08-31,8FH1DDY73A,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Lưu kho,'13/08/2020 14:36:37 - GHN-PFA1A0|THAY ĐỔI THÔ...,8,1,0,Kho giao,2020-08-23 20:23:45,2020-08-28 20:23:45,2 days 21:10:01.996709,18 days 02:54:28.387993,nan,Kết thúc
2020-08-31,8FHSAHAY5K,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Giao hàng không thành công,'17/08/2020 06:39:33 - GHN-DFC1A1|Khách đổi đị...,1,1,0,Kho giao,2020-08-16 14:17:02,2020-08-21 14:17:02,10 days 03:16:44.996709,17 days 20:56:48.387993,nan,Kết thúc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-31,8FHY6Q9H4F,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Chuyển hoàn,'28/08/2020 04:47:55 - GHN-DFC1A7|THAY ĐỔI THÔ...,0,1,0,Kho giao luân chuyển,2020-08-28 13:33:16,2020-08-30 01:33:16,1 days 16:00:31.279446,4 days 04:48:49.333304,Shopee,Khởi tạo
2020-08-31,8FHAQYD35A,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Chuyển hoàn,'28/08/2020 03:18:06 - GHN-DFC1A7|THAY ĐỔI THÔ...,0,1,0,Kho giao luân chuyển,2020-08-28 16:34:26,2020-08-31 00:34:26,0 days 16:59:21.279446,4 days 06:16:02.333304,Shopee,Khởi tạo
2020-08-31,31416789163172,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Chuyển hoàn,'29/08/2020 10:42:22 - GHN-DFC1A7|THAY ĐỔI THÔ...,nan,1,0,Kho giao luân chuyển,2020-08-29 14:53:09,2020-08-31 02:53:09,0 days 14:40:38.279446,4 days 04:57:39.333304,Tiki,Kết thúc
2020-08-31,8FHHF9QSDD,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Chuyển hoàn,'27/08/2020 10:56:23 - GHN-PFA1A0|THAY ĐỔI THÔ...,2,2,0,Kho giao luân chuyển,2020-08-29 19:49:07,2020-08-31 07:49:07,0 days 09:44:40.279446,4 days 03:50:29.387993,nan,Kết thúc


In [10]:
# from datetime import datetime

# dt = datetime.strptime('08/08/2020 14:10:12', '%d/%m/%Y %H:%M:%S')
# print(dt)
# def iter_df(df):
#     """Generator for df."""
#     for val in df.columns:
#         yield val
#     for row in df.to_numpy():
#         for val in row:
#             if pd.isna(val):
#                 yield ''
#             else:
#                 yield val

In [11]:
# to excel
# data_to_excel = data[[
#     'MaDH',
#     'KhoHienTai',
#     'TrangThai',
#     'GhiChuGHN',
#     'SoLanLay',
#     'SoLanGiao',
#     'SoLanTra',
#     'LoaiBacklog',
#     'N0',
#     'N+',
#     'Aging',
#     'Aging (hours)',
#     'e-commerces',
# ]]

# data_to_excel.to_excel('backlog.excel', sheet='Sheet1')